In [ ]:
# Purpose: Apply a number of classifiers that we trained on the labeled Kaggle data from the 1st Republican debate,
#          August 2015, to a sample of election tweets. Look at the results.
#         Compare to a few hundred manually classified election tweets. Calculate accuracy and Kappa.
#Author:  Carol Sniegoski
#Date:  April 28, 2016
#Course: MAS DSE Capstone, Spring 2016

In [57]:
import pandas as pd
pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import numpy as np

from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import classification

print "done"

 done


In [55]:
# Function defs.

# Return string with non-ascii chars removed.
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

# Get class frequencies. Input is a series.
def get_class_frequencies(s):
    counts = s.value_counts()
    total = counts.sum()
    counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )
    return counts.sort_index()

# From newer version of python.
def cohen_kappa_score(y1, y2, labels=None, weights=None):
    confusion = metrics.confusion_matrix(y1, y2, labels=labels)
    n_classes = confusion.shape[0]
    sum0 = np.sum(confusion, axis=0)
    sum1 = np.sum(confusion, axis=1)
    expected = np.outer(sum0, sum1) / np.sum(sum0)

    if weights is None:
        w_mat = np.ones([n_classes, n_classes], dtype=np.int)
        w_mat.flat[:: n_classes + 1] = 0
    elif weights == "linear" or weights == "quadratic":
        w_mat = np.zeros([n_classes, n_classes], dtype=np.int)
        w_mat += np.arange(n_classes)
        if weights == "linear":
            w_mat = np.abs(w_mat - w_mat.T)
        else:
            w_mat = (w_mat - w_mat.T) ** 2
    else:
        raise ValueError("Unknown kappa weighting type.")

    k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
    return 1 - k

print('done')

 done


In [8]:
# Locate data dir.
%ls ../data/sentiment/

Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv      debate08_sentiment_tweets.tsv
Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv.bak  debate08_sentiment_tweets.tsv.zip
bing-liu_opinion-lexicon-English_d04-13-16.rar                      opinion-lexicon-English.rar


In [29]:
# Define variables.

clean_text = 'ascii_clean'
data_dir = "../data/sentiment/"
datafile_name = "Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv"
datafile = data_dir + datafile_name
print datafile

../data/sentiment/Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv


In [12]:
# Load datafile.
df = pd.read_csv(datafile)
#df=df.rename(columns = {'label':'label_LinearSVC_char3grams_0.75sample'})
print df.shape
df.head(2)

(1859, 6)


,Unnamed: 0,text,ascii,ascii_clean,label_LinearSVC_char3grams_0.75sample,label_cas_04-27-16
0,0,"RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/…","RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/",rt @ggreenwald clinton sadly never complained about overclassification run amok when whistleblowers sources were prosecuted... url,Negative,Negative
1,1,Father: Ted Cruz’s success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,Father: Ted Cruzs success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,father ted cruzs success depends on people of faith url url,Negative,Neutral


In [43]:
# Locate the files containing the trained classifiers and featurizers.
%ls ../classification/
dir = '../classification/'

LinearSVC_char3grams_0.75randomsample_2016-04-27.pkl             RandomForest-d5_char3grams_originaldata_2016-04-28.pkl_98.npy
LinearSVC_char3grams_0.75randomsample_2016-04-27.pkl_01.npy      RandomForest-d5_char3grams_originaldata_2016-04-28.pkl_99.npy
LinearSVC_char3grams_0.75randomsample_2016-04-27.pkl_02.npy      RandomForest_char3grams_0.75randomsample_2016-04-28.pkl
LinearSVC_char3grams_0.75randomsample_2016-04-27.pkl_03.npy      RandomForest_char3grams_0.75randomsample_2016-04-28.pkl_01.npy
LinearSVC_char3grams_originaldata_2016-04-28.pkl                 RandomForest_char3grams_0.75randomsample_2016-04-28.pkl_02.npy
LinearSVC_char3grams_originaldata_2016-04-28.pkl_01.npy          RandomForest_char3grams_0.75randomsample_2016-04-28.pkl_03.npy
LinearSVC_char3grams_originaldata_2016-04-28.pkl_02.npy          RandomForest_char3grams_0.75randomsample_2016-04-28.pkl_04.npy
RandomForest-d5_char3grams_originaldata_2016-04-28.pkl           RandomForest_char3grams_0.75randomsample_2016-04

In [60]:
# Load the fitted featurizers.
#f_names = [ 'char3grams_0.75randomsample_2016-04-27',
#           'char3grams_originaldata_2016-04-28',
#           'char3grams_0.75randomsample_2016-04-28']
f_names = { 'char3grams_1.8randomsample_2016-04-29'}
featurizers = {}

for f_name in f_names:
    filename = 'featurizer_' + f_name + '.pkl'
    file = dir + filename
    print file
    featurizers[f_name] = joblib.load(file)
    print f_name, type(featurizers[f_name])

#featurizer = joblib.load(file)
#print type(featurizer)

print 'done'

../classification/featurizer_char3grams_1.8randomsample_2016-04-29.pkl
char3grams_1.8randomsample_2016-04-29 <class 'sklearn.feature_extraction.text.CountVectorizer'>
done


In [61]:
# Load the trained classifiers.
dir = "../classification/"
#c_names = [ ('LinearSVC', 'char3grams_0.75randomsample_2016-04-27'),
#          ( 'LinearSVC', 'char3grams_originaldata_2016-04-28'),
#          ( 'RandomForest', 'char3grams_0.75randomsample_2016-04-28'),
#          ( 'RandomForest-d5', 'char3grams_originaldata_2016-04-28')]
c_names = { ('LinearSVC', 'char3grams_1.8randomsample_2016-04-29')}
classifiers = {}

for c_name in c_names:
    filename = c_name[0] + "_" + c_name[1] + '.pkl'
    #filename = "LinearSVC_char3grams_0.75randomsample_2016-04-27.pkl"
    file = dir + filename
    print file
    classifiers[c_name] = joblib.load(file)
    print c_name, type(classifiers[c_name])
    
#classifier = joblib.load(file)
#print type(classifier)

../classification/LinearSVC_char3grams_1.8randomsample_2016-04-29.pkl
('LinearSVC', 'char3grams_1.8randomsample_2016-04-29') <class 'sklearn.svm.classes.LinearSVC'>


In [62]:
# Featurize the cleaned text data.

# Convert to ndarray.
#X_samples.append(df_sample[clean_text].values)
#this_feature_vector_list.append( vectorizer.fit_transform(X_sample) )

# Don't do this - it puts the entire feature matrix into each X_feauturized value.
#df['X_featurized'] = featurizer.transform( df[clean_text].values )
#df.drop('X_featurized', axis=1, inplace=True)

X_feature_matrices = {}

for key, val in featurizers.items():
    print key, type(val)
    X_feature_matrices[key] = val.transform( df[clean_text].values )
    print X_feature_matrices[key].shape


#X_featurized = featurizer.transform( df[clean_text].values )
print 'done'

#print df.shape
#print type(X_featurized)
#print X_featurized.shape

char3grams_1.8randomsample_2016-04-29 <class 'sklearn.feature_extraction.text.CountVectorizer'>
(1859, 9393)
done


In [63]:
# Classify.

truth = 'label_cas_04-27-16'
label_col_names = []

for key, val in classifiers.items():
    print key, type(val)
    col_name = 'label_' + key[0] + '_' + key[1]
    df[col_name] = val.predict(X_feature_matrices[key[1]])
    label_col_names.append(col_name)

print 'done'
print label_col_names
df.head(2)
    
#y_preds = classifier.predict(X_featurized)
#print 'done'
#print type( y_preds )
#print y_preds.shape

('LinearSVC', 'char3grams_1.8randomsample_2016-04-29') <class 'sklearn.svm.classes.LinearSVC'>
done
['label_LinearSVC_char3grams_1.8randomsample_2016-04-29']


,Unnamed: 0,text,ascii,ascii_clean,label_LinearSVC_char3grams_0.75sample,label_cas_04-27-16,label_LinearSVC_char3grams_0.75randomsample_2016-04-27,label_RandomForest_char3grams_0.75randomsample_2016-04-28,label_LinearSVC_char3grams_originaldata_2016-04-28,label_RandomForest-d5_char3grams_originaldata_2016-04-28,label_LinearSVC_char3grams_1.8randomsample_2016-04-29
0,0,"RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/…","RT @ggreenwald: Clinton sadly never complained about ""overclassification run amok"" when whistleblowers &amp; sources were prosecuted... https:/",rt @ggreenwald clinton sadly never complained about overclassification run amok when whistleblowers sources were prosecuted... url,Negative,Negative,Negative,Negative,Negative,Negative,Negative
1,1,Father: Ted Cruz’s success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,Father: Ted Cruzs success depends on people of faith https://t.co/sXviGD11Xw https://t.co/T0cdqrNowK,father ted cruzs success depends on people of faith url url,Negative,Neutral,Negative,Neutral,Negative,Negative,Neutral


In [69]:
# Reset column names.
print df.columns.values

label_col_names = [ 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27', 
                   'label_RandomForest_char3grams_0.75randomsample_2016-04-28',
                   'label_LinearSVC_char3grams_originaldata_2016-04-28',
                   'label_RandomForest-d5_char3grams_originaldata_2016-04-28',
                   'label_LinearSVC_char3grams_1.8randomsample_2016-04-29' ]
print 'done'

['Unnamed: 0' 'text' 'ascii' 'ascii_clean'
 'label_LinearSVC_char3grams_0.75sample' 'label_cas_04-27-16'
 'label_LinearSVC_char3grams_0.75randomsample_2016-04-27'
 'label_RandomForest_char3grams_0.75randomsample_2016-04-28'
 'label_LinearSVC_char3grams_originaldata_2016-04-28'
 'label_RandomForest-d5_char3grams_originaldata_2016-04-28'
 'label_LinearSVC_char3grams_1.8randomsample_2016-04-29']
done


In [70]:
# Look at class proportions.

for label_col_name in [truth]+label_col_names:
    f = get_class_frequencies(df[label_col_name])
    print label_col_name
    print f
    print
    
print 'done'
    
#f1 = get_class_frequencies(df['label_cas_04-27-16'])
#f2 = get_class_frequencies(df['label'])
#print f1
#print f2

label_cas_04-27-16
Negative    (Count: 98, Frequency: 0.52)
Neutral     (Count: 55, Frequency: 0.29)
Positive    (Count: 37, Frequency: 0.19)
dtype: object

label_LinearSVC_char3grams_0.75randomsample_2016-04-27
Negative    (Count: 671, Frequency: 0.36)
Neutral     (Count: 783, Frequency: 0.42)
Positive    (Count: 405, Frequency: 0.22)
dtype: object

label_RandomForest_char3grams_0.75randomsample_2016-04-28
Negative    (Count: 696, Frequency: 0.37)
Neutral     (Count: 898, Frequency: 0.48)
Positive    (Count: 265, Frequency: 0.14)
dtype: object

label_LinearSVC_char3grams_originaldata_2016-04-28
Negative    (Count: 915, Frequency: 0.49)
Neutral     (Count: 653, Frequency: 0.35)
Positive    (Count: 291, Frequency: 0.16)
dtype: object

label_RandomForest-d5_char3grams_originaldata_2016-04-28
Negative    (Count: 1859, Frequency: 1.00)
dtype: object

label_LinearSVC_char3grams_1.8randomsample_2016-04-29
Negative    (Count: 780, Frequency: 0.42)
Neutral     (Count: 768, Frequency: 0.41)
Pos

In [71]:
# Look at metrics for those rows that have a 'truth' label.

y = df[df[truth].notnull()][truth].values
for label_col_name in label_col_names:
    y_pred = df[df[truth].notnull()][label_col_name].values
    
    print label_col_name
    print y.shape 
    print y_pred.shape 
    
    print "accuracy =", metrics.accuracy_score(y, y_pred) 
    print "cohen's kappa =", cohen_kappa_score(y, y_pred)
    print ' Neg', ' Neu', ' Pos' 
    print metrics.confusion_matrix(y, y_pred) 
    print metrics.classification_report(y, y_pred) 
    print

print 'done'

#y = df[df['label_cas_04-27-16'].notnull()]['label_cas_04-27-16'].values
#y_pred = df[df['label_cas_04-27-16'].notnull()]['label'].values


# Reverse y to make sure it hurts the scores!
#y = y[::-1]




label_LinearSVC_char3grams_0.75randomsample_2016-04-27
(190,)
(190,)
accuracy = 0.5
cohen's kappa = 1
 Neg  Neu  Pos
[[50 32 16]
 [15 30 10]
 [ 9 13 15]]
             precision    recall  f1-score   support

   Negative       0.68      0.51      0.58        98
    Neutral       0.40      0.55      0.46        55
   Positive       0.37      0.41      0.38        37

avg / total       0.54      0.50      0.51       190


label_RandomForest_char3grams_0.75randomsample_2016-04-28
(190,)
(190,)
accuracy = 0.436842105263
cohen's kappa = 1
 Neg  Neu  Pos
[[52 40  6]
 [23 28  4]
 [16 18  3]]
             precision    recall  f1-score   support

   Negative       0.57      0.53      0.55        98
    Neutral       0.33      0.51      0.40        55
   Positive       0.23      0.08      0.12        37

avg / total       0.43      0.44      0.42       190


label_LinearSVC_char3grams_originaldata_2016-04-28
(190,)
(190,)
accuracy = 0.415789473684
cohen's kappa = 1
 Neg  Neu  Pos
[[57 34  7]
 [31

In [75]:
# Locate output dir.
%ls ../data/sentiment

Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv      debate08_sentiment_tweets.tsv
Labeled_electionTweets_randomSample_allPartitions_04-26-16.csv.bak  debate08_sentiment_tweets.tsv.zip
bing-liu_opinion-lexicon-English_d04-13-16.rar                      opinion-lexicon-English.rar


In [78]:
# Write to csv.

out_dir = "../data/sentiment/"
outfile_name = "Labeled_electionTweets_randomSample_allPartitions_04-30-16.csv"
outfile = out_dir + "Labeled_" + outfile_name
print outfile
df.to_csv(outfile)
print 'done'

../data/sentiment/Labeled_Labeled_electionTweets_randomSample_allPartitions_04-30-16.csv
done
